In [ ]:
#Load Module
import os
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import time
import copy


In [ ]:
# Get files name
path1 = 'train1/'
lenght = 47

sg_files = []
spike_files = []

for i in range(1, lenght):
    sg_files.append( path1 + str(i) + '_sg.csv')
    spike_files.append( path1 + str(i) + '_spike.csv')
    
print(sg_files)
print(spike_files)


In [ ]:
# Read file and seperate each feature to do data preprocessing
s = time.time()
print('Concating sg...')
sg_data = [pd.read_csv(i) for i in sg_files]

print('Concating spike')
spike_data = [pd.read_csv(i) for i in spike_files]

print('Reading MaxWear')
output = pd.read_csv(path1+'00_Wear_data.csv')
output.drop(['Index'], axis="columns", inplace=True)

print('sg_data: ' + str(len(sg_data)))
print('spike_data: ' + str(len(spike_data)))

print('cost time:', time.time() - s)


In [13]:
# Define feature index
Sg_Feature_Index = ['A', 'B', 'C', 'D', 'E', 'F', 'H', 'I']
Spike_Feature_Index = ['A', 'B', 'C', 'D']

s = time.time()

# Concat data
sg_data_concat = pd.concat(sg_data, axis=1)
spike_data_concat = pd.concat(spike_data, axis=1)

print('sg_data_concat' + str(sg_data_concat.shape))
print('spike_data_concat' + str(spike_data_concat.shape))

# Create dict then use the last value of that column to fill in to get a same dimention in one featrue(column)
sg = {}
spike = {}

for i in Sg_Feature_Index:
  print('fillna sg_' + str(i) + '...')
  data = sg_data_concat[i].copy()
  for j in range(0, lenght-1):
      initial_data = data.iloc[:, j]
      if True in initial_data.isnull().values:
          last_value = initial_data[np.where(
              initial_data.isnull().values == True)[0][0]-1]
          data.iloc[:, j] = sg_data_concat[i].iloc[:, j].fillna(last_value)
          sg_data_concat[i] = data
  sg['sg_' + str(i)] = sg_data_concat[i]

for i in Spike_Feature_Index:
  print('fillna spike_' + str(i) + '...')
  data = spike_data_concat[i].copy()
  for j in range(0, lenght-1):
      initial_data = data.iloc[:, j]
      if True in initial_data.isnull().values:
          last_value = initial_data[np.where(
              initial_data.isnull().values == True)[0][0]-1]
          data.iloc[:, j] = spike_data_concat[i].iloc[:, j].fillna(last_value)
          spike_data_concat[i] = data
  spike['spike_' + str(i)] = spike_data_concat[i]

print('cost time:', time.time() - s)

print(sg.keys())
print(spike.keys())

# np.save()
sg_feture = list(sg.keys())
spike_feture = list(spike.keys())

sg_data_concat(416001, 414)
spike_data_concat(1040001, 184)
fillna sg_A...
fillna sg_B...
fillna sg_C...
fillna sg_D...
fillna sg_E...
fillna sg_F...
fillna sg_H...
fillna sg_I...
fillna spike_A...
fillna spike_B...
fillna spike_C...
fillna spike_D...
cost time: 277.23595786094666


In [16]:
spike['spike_D']

,D,D,D,D,D,D,D,D,D,D,...,D,D,D,D,D,D,D,D,D,D
0,0.833795,2.907015,0.878865,2.952085,1.284495,2.726735,2.095755,3.087295,4.529535,3.132365,...,1.014075,3.357715,3.808415,2.771805,2.681665,2.636595,2.726735,2.411245,2.546455,2.591525
1,1.509845,2.005615,1.239425,3.402785,0.383095,3.087295,3.087295,2.005615,4.078835,3.537995,...,2.726735,3.402785,3.673205,2.997155,2.411245,3.132365,2.952085,2.366175,2.366175,2.140825
2,0.878865,2.681665,1.464775,4.259115,1.690125,3.492925,2.050685,2.546455,4.123905,3.537995,...,2.952085,4.799955,4.349255,2.050685,2.952085,3.988695,3.177435,2.005615,3.402785,3.853485
3,2.952085,2.861945,2.050685,2.276035,1.599985,3.402785,2.907015,3.222505,3.853485,3.628135,...,1.329565,3.402785,4.484465,2.907015,2.185895,2.140825,1.870405,1.780265,2.861945,2.411245
4,3.132365,2.681665,2.411245,2.907015,1.825335,2.861945,2.952085,1.960545,2.997155,3.898555,...,2.501385,3.222505,1.915475,2.591525,1.960545,2.681665,2.230965,2.140825,0.923935,1.960545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039996,0.383095,-0.338025,0.518305,0.247885,0.157745,0.383095,0.608445,0.022535,0.878865,0.653515,...,0.157745,0.247885,0.969005,1.419705,0.743655,0.563375,-0.022535,0.157745,-0.292955,1.059145
1039997,0.247885,0.157745,-0.608445,0.202815,-0.022535,0.112675,1.059145,0.969005,0.878865,0.653515,...,0.157745,0.247885,0.969005,1.419705,0.743655,0.563375,-0.022535,0.157745,-0.292955,1.059145
1039998,-0.292955,-0.473235,0.833795,0.292955,0.878865,1.059145,-0.563375,0.157745,-0.067605,0.428165,...,0.157745,0.247885,0.969005,1.419705,0.743655,0.563375,-0.022535,0.157745,-0.292955,1.059145
1039999,0.022535,-0.112675,0.202815,-0.653515,-0.743655,0.338025,-0.022535,0.473235,-0.383095,1.149285,...,0.157745,0.247885,0.969005,1.419705,0.743655,0.563375,-0.022535,0.157745,-0.292955,1.059145
